In [1]:
import cupy as cp
import cupyx
import scanpy as sc
import spatialleiden as sl
import squidpy as sq
import numpy as np
from cupyx.scipy.sparse import csr_matrix
import os
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
import random
import pandas as pd

In [11]:
pathout = '/data/kanferg/Sptial_Omics/projects/NguyenLab/spatialomicstoolkit/out_1'
adata_concat = sc.read_h5ad(os.path.join(pathout, "andata_filter_logNorm_hvg_leiden_harmony_scvi_con_mouse_cov_umap.h5ad"))

In [18]:
del_obs = [ 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts','segmentation_method', 'region', 'z_level', 'cell_labels','n_genes_by_counts', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'outlier', 'cluster', '_scvi_batch', '_scvi_labels', 'cluster_scVI', 'cluster_cell2loc']
del_var = [ 'total_counts', 'mean_counts', 'pct_dropout_by_counts', 'log1p_total_counts', 'log1p_mean_counts', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm']
del_uns = [ '_scvi_manager_uuid', '_scvi_uuid', 'before_int', 'cell2locNigb', 'hvg', 'leiden', 'log1p', 'neighbors', 'pca', 'q05_cell_abundance_w_sf', 'scVInigb', 'umap']
del_obsm = ['Unintegrated_umap', 'X_cell2loc', 'X_pca','X_pca_before', 'X_scVI', '_scvi_extra_categorical_covs', 'cell2lo_umap', 'harmony_umap', 'scVI_umap']
del_obsp = ['before_int_connectivities', 'before_int_distances', 'cell2locNigb_connectivities', 'cell2locNigb_distances', 'connectivities', 'distances', 'scVInigb_connectivities', 'scVInigb_distances']
for rem in del_obs:
    if rem in adata_concat.obs:  # Ensure the key exists before deleting
        del adata_concat.obs[rem]

# Deleting items from adata_concat.var
for rem in del_var:
    if rem in adata_concat.var:  # Ensure the key exists before deleting
        del adata_concat.var[rem]

# Deleting items from adata_concat.uns
for rem in del_uns:
    if rem in adata_concat.uns:  # Ensure the key exists before deleting
        del adata_concat.uns[rem]

# Deleting items from adata_concat.obsm
for rem in del_obsm:
    if rem in adata_concat.obsm:  # Ensure the key exists before deleting
        del adata_concat.obsm[rem]

# Deleting items from adata_concat.obsp
for rem in del_obsp:
    if rem in adata_concat.obsp:  # Ensure the key exists before deleting
        del adata_concat.obsp[rem]

In [20]:
adata_concat.X = adata_concat.layers['log']

In [26]:
adata_concat

AnnData object with n_obs × n_vars = 664093 × 2000
    obs: 'cell_id', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'batch', 'cluster_Harmony'
    var: 'n_cells_by_counts'
    uns: 'HarmonyNigb', 'spatial_neighbors'
    obsm: 'X_pca_Harmony', 'spatial'
    varm: 'PCs'
    layers: 'counts', 'log', 'scvi_normalized'
    obsp: 'HarmonyNigb_connectivities', 'HarmonyNigb_distances', 'spatial_connectivities', 'spatial_distances'

In [27]:
adata_concat.obsp['connectivities'] = adata_concat.obsp['HarmonyNigb_connectivities']
adata_concat.obsp['distances'] = adata_concat.obsp['HarmonyNigb_distances']

In [22]:
sq.gr.spatial_neighbors(adata_concat, coord_type="generic")

In [28]:
adata_concat.obsp["spatial_connectivities"] = sl.distance2connectivity(adata_concat.obsp["spatial_distances"])

In [29]:
seed = 42
sl.spatialleiden(adata_concat, layer_ratio=1.5, directed=(False, True), seed=seed)

In [30]:
adata_concat

AnnData object with n_obs × n_vars = 664093 × 2000
    obs: 'cell_id', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'batch', 'cluster_Harmony', 'spatialleiden'
    var: 'n_cells_by_counts'
    uns: 'HarmonyNigb', 'spatial_neighbors'
    obsm: 'X_pca_Harmony', 'spatial'
    varm: 'PCs'
    layers: 'counts', 'log', 'scvi_normalized'
    obsp: 'HarmonyNigb_connectivities', 'HarmonyNigb_distances', 'spatial_connectivities', 'spatial_distances', 'connectivities', 'distances'

In [31]:
andata_save = adata_concat.copy()
andata_save.write_h5ad(os.path.join(pathout, "andata_filter_logNorm_hvg_leiden_harmony_spatialleiden.h5ad"))